In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import math as mt

In [ ]:
M = 0.8 #change later
reference_fuel_consumption = 0.7296 #who knows
reference_nox = 19.1835 #IDGAF
number_of_engines = 2 
temp_sea_level = 288.15
pres_sea_level = 101325
gamma = 1.4
R = 287.05


In [ ]:
#need to streamline it, combine datasets into one

echam = 
accf = 
contrail =

In [ ]:
#change the datasets so it correspomds with above code

velocity = M*np.sqrt(gamma*R*echam['tm1'])
delta = echam['tm1']*(1+0.2*M**2)**3.5 / temp_sea_level
theta = echam['press']*(1+0.2*M**2)**3.5 / pres_sea_level
f_a_i = reference_fuel_consumption*delta*np.sqrt(theta)
f_cr = f_a_i*number_of_engines
H_c = np.exp(-19*(echam['qm1']-0.00634))
EINO_x = reference_nox*delta**0.4*theta**0.3*H_c
co2_atr20_perkm = accf['atr20_co2']/velocity*f_cr
h2o_atr20_perkm = accf['atr20_h2o']/velocity*f_cr
o3_atr20_perkm = accf['atr20_o3']/velocity*f_cr*EINO_x*10**(-3)
ch4_atr20_perkm = accf['atr20_ch4']/velocity*f_cr*EINO_x*10**(-3)
contr_atr20_perkm = accf['atr20_contrail']*contrail['potcov']

